# FluidityNonlocal Flow Curve: Shear Banding Detection

## Learning Objectives

1. Understand spatial diffusion via cooperativity length ξ in thixotropic fluids
2. Detect shear banding from fluidity profiles (CV > 0.3, f_max/f_min > 10)
3. Compare local vs non-local model predictions on emulsion flow curves
4. Fit 10-parameter nonlocal model using NLSQ → NUTS workflow
5. Quantify when non-local effects dominate vs local approximation suffices

## Model Overview

**FluidityNonlocal** extends the local fluidity model with spatial diffusion:

$$
\frac{\partial f}{\partial t} = \frac{f_{\text{loc}}(\sigma) - f}{\theta} + \xi^2 \frac{\partial^2 f}{\partial y^2}
$$

where:
- $f_{\text{loc}}(\sigma) = f_0 + (f_1 - f_0) \tanh\left(\frac{\sigma - \sigma_c}{\Delta\sigma}\right)$ (local steady-state solution)
- $\theta$ = structural relaxation time
- $\xi$ = cooperativity length (spatial diffusion scale)
- $y$ = gap coordinate (0 to $h$)

**Shear Banding Metrics:**
- Coefficient of variation: $\text{CV} = \sigma(f) / \mu(f) > 0.3$
- Fluidity contrast: $f_{\max} / f_{\min} > 10$

**Parameters (10):**
1. $f_0$ = low-stress fluidity (s⁻¹)
2. $f_1$ = high-stress fluidity (s⁻¹)
3. $\sigma_c$ = critical stress (Pa)
4. $\Delta\sigma$ = transition width (Pa)
5. $\theta$ = relaxation time (s)
6. $n$ = flow index
7. $\alpha$ = stress exponent
8. $K$ = consistency (Pa·sⁿ)
9. $\tau_y$ = yield stress (Pa)
10. $\xi$ = cooperativity length (m)

In [ ]:
# Colab setup
try:
    import google.colab
    IN_COLAB = True
    !pip install -q rheojax nlsq numpyro arviz
except ImportError:
    IN_COLAB = False

# Standard imports
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# RheoJAX imports
from rheojax.core.jax_config import safe_import_jax
from rheojax.core.data import RheoData
from rheojax.models.fluidity import FluidityNonlocal, FluidityLocal
from rheojax.logging import configure_logging, get_logger
from rheojax.utils.metrics import compute_fit_quality

# JAX setup (NLSQ auto-configures float64)
jax, jnp = safe_import_jax()

# Logging
configure_logging(level="INFO")
logger = get_logger(__name__)

# Plotting defaults
plt.rcParams.update({
    'figure.figsize': (10, 6),
    'font.size': 11,
    'axes.labelsize': 12,
    'axes.titlesize': 13,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'legend.fontsize': 10,
})

logger.info("Setup complete", jax_version=jax.__version__)

In [ ]:
def compute_fit_quality(y_true, y_pred):
    """Compute R² and RMSE."""
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    residuals = y_true - y_pred
    if y_true.ndim > 1:
        residuals = residuals.ravel()
        y_true = y_true.ravel()
    ss_res = np.sum(residuals**2)
    ss_tot = np.sum((y_true - np.mean(y_true))**2)
    r2 = 1.0 - ss_res / ss_tot if ss_tot > 0 else 0.0
    rmse = np.sqrt(np.mean(residuals**2))
    return {'R2': r2, 'RMSE': rmse}

## Theory: Non-Local Effects and Shear Banding

### Spatial Diffusion PDE

The fluidity field $f(y, t)$ evolves according to:

$$
\frac{\partial f}{\partial t} = \underbrace{\frac{f_{\text{loc}}(\sigma) - f}{\theta}}_{\text{Local relaxation}} + \underbrace{\xi^2 \frac{\partial^2 f}{\partial y^2}}_{\text{Spatial diffusion}}
$$

**Cooperativity Length $\xi$:**
- Physical interpretation: length scale over which stress perturbations propagate
- Typical values: 1-100 μm for soft materials
- $\xi \ll h$: local approximation valid (FluidityLocal sufficient)
- $\xi \sim h$: non-local effects important (shear banding possible)

### Shear Banding Criterion

Steady-state fluidity profile $f(y)$ indicates banding when:

1. **High spatial variation:**
   $$\text{CV} = \frac{\sigma_f}{\mu_f} > 0.3$$
   where $\sigma_f = \sqrt{\langle (f - \langle f \rangle)^2 \rangle}$

2. **Strong contrast:**
   $$\frac{f_{\max}}{f_{\min}} > 10$$

3. **Bimodal distribution:**
   - Low-fluidity band: $f \approx f_0$ (nearly solid)
   - High-fluidity band: $f \approx f_1$ (flowing)

### Constitutive Relation

Stress-fluidity coupling:

$$
\sigma = \tau_y + K \dot{\gamma}^n, \quad \dot{\gamma} = f \cdot \sigma^\alpha
$$

**Key Difference from Local Model:**
- Local: $f$ uniform across gap → single shear rate
- Nonlocal: $f(y)$ profile → shear rate banding $\dot{\gamma}(y)$

## Data: Emulsion Flow Curve

Synthetic emulsion data showing characteristic stress plateau (yield region).

In [ ]:
# Generate synthetic emulsion flow curve data
np.random.seed(42)

# Shear rate range spanning yield transition
gamma_dot = np.logspace(-3, 2, 50)  # 0.001 to 100 s^-1

# Ground truth parameters (moderate shear banding)
f0_true = 0.01      # Low fluidity (s^-1)
f1_true = 1.0       # High fluidity (s^-1)
sigma_c_true = 50.0 # Critical stress (Pa)
delta_sigma_true = 10.0  # Transition width (Pa)
theta_true = 5.0    # Relaxation time (s)
n_true = 0.5        # Shear-thinning
alpha_true = 1.0    # Linear stress dependence
K_true = 20.0       # Consistency (Pa·s^n)
tau_y_true = 30.0   # Yield stress (Pa)
xi_true = 5e-5      # 50 μm cooperativity length

# Simple Herschel-Bulkley flow curve for synthetic data
# sigma = tau_y + K * gamma_dot^n
sigma_true = tau_y_true + K_true * gamma_dot**n_true

# Add measurement noise (5% relative + 2 Pa absolute)
noise = 0.05 * sigma_true + 2.0 * np.random.randn(len(sigma_true))
sigma = sigma_true + noise

# Ensure positive stresses
sigma = np.maximum(sigma, 1.0)

# Visualization
fig, ax = plt.subplots()
ax.loglog(gamma_dot, sigma, 'o', label='Measured', alpha=0.7)
ax.loglog(gamma_dot, sigma_true, '-', label='True (no noise)', linewidth=2)
ax.axhline(sigma_c_true, color='gray', linestyle='--', label=f'σ_c = {sigma_c_true} Pa')
ax.set_xlabel('Shear Rate $\dot{\gamma}$ (s$^{-1}$)')
ax.set_ylabel('Stress $\sigma$ (Pa)')
ax.set_title('Emulsion Flow Curve (Synthetic Data)')
ax.legend()
ax.grid(True, which='both', alpha=0.3)
plt.tight_layout()
plt.show()
plt.close('all')

logger.info("Data generated", n_points=len(gamma_dot), sigma_range=(sigma.min(), sigma.max()))

## Model Initialization

Configure **FluidityNonlocal** with spatial discretization:
- `N_y = 64`: number of grid points across gap
- `gap_width = 1e-3`: 1 mm gap (typical rheometer geometry)

In [ ]:
# Create RheoData object
rheo_data = RheoData(
    x=gamma_dot,
    y=sigma,
    initial_test_mode='flow_curve',
    metadata={'material': 'emulsion', 'temperature': 25.0}
)

# Initialize nonlocal model
model_nonlocal = FluidityNonlocal(
    N_y=64,           # Spatial resolution
    gap_width=1e-3    # 1 mm gap
)

logger.info(
    "Model initialized",
    n_params=len(model_nonlocal.parameters),
    N_y=64,
    gap_width=1e-3
)

# Display parameter bounds
print("\nParameter Bounds:")
for name, param in model_nonlocal.parameters.items():
    print(f"  {name:12s}: [{param.bounds[0]:8.2e}, {param.bounds[1]:8.2e}]")

## NLSQ Fitting

Non-linear least squares optimization using NLSQ 0.6.6+ workflow system.

In [ ]:
# NLSQ fit (warm-start for Bayesian inference)
model_nonlocal.fit(gamma_dot, sigma, test_mode='flow_curve', method='scipy')

# Extract fitted parameters
params_nlsq = {
    name: model_nonlocal.parameters[name].value
    for name in model_nonlocal.parameters.keys()
}

print("\nNLSQ Fitted Parameters:")
for name, value in params_nlsq.items():
    print(f"  {name:12s}: {value:10.4e}")

# Predictions
sigma_pred_nlsq = model_nonlocal.predict(gamma_dot, test_mode='flow_curve')

# Compute fit quality
metrics = compute_fit_quality(sigma, np.array(sigma_pred_nlsq).flatten())
r_squared = metrics['R2']
rmse = metrics['RMSE']

print(f"\nR² = {r_squared:.6f}")
print(f"RMSE = {rmse:.4f} Pa")

# Plot fit quality
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Flow curve
ax1.loglog(gamma_dot, sigma, 'o', label='Data', alpha=0.7)
ax1.loglog(gamma_dot, sigma_pred_nlsq, '-', label='NLSQ Fit', linewidth=2)
ax1.set_xlabel(r'Shear Rate $\dot{\gamma}$ (s$^{-1}$)')
ax1.set_ylabel(r'Stress $\sigma$ (Pa)')
ax1.set_title(f'NLSQ Fit (R² = {r_squared:.4f})')
ax1.legend()
ax1.grid(True, which='both', alpha=0.3)

# Residuals
residuals = sigma - np.array(sigma_pred_nlsq).flatten()
ax2.semilogx(gamma_dot, residuals, 'o', alpha=0.7)
ax2.axhline(0, color='k', linestyle='--', linewidth=1)
ax2.set_xlabel(r'Shear Rate $\dot{\gamma}$ (s$^{-1}$)')
ax2.set_ylabel(r'Residual $\sigma - \sigma_{\mathrm{pred}}$ (Pa)')
ax2.set_title('Residual Analysis')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()
plt.close('all')

logger.info("NLSQ fit complete", R2=r_squared, RMSE=rmse)

## Bayesian Inference with NUTS

Use NLSQ solution as warm-start for Hamiltonian Monte Carlo sampling.

In [ ]:
# Bayesian inference (4 chains for production-ready diagnostics)
result_bayes = model_nonlocal.fit_bayesian(
    gamma_dot, sigma,
    test_mode='flow_curve',
    num_warmup=1000,
    num_samples=2000,
    num_chains=4,
    seed=42
)

# Extract posterior samples
posterior = result_bayes.posterior_samples

# Compute credible intervals (95%)
intervals = model_nonlocal.get_credible_intervals(posterior, credibility=0.95)

print("\nBayesian Parameter Estimates (95% HDI):")
print(f"{'Parameter':<12s} {'NLSQ':>12s} {'Median':>12s} {'Lower':>12s} {'Upper':>12s}")
print("-" * 60)
for name in model_nonlocal.parameters.keys():
    nlsq_val = params_nlsq[name]
    median = float(jnp.median(posterior[name]))
    lower, upper = intervals[name]
    print(f"{name:<12s} {nlsq_val:12.4e} {median:12.4e} {lower:12.4e} {upper:12.4e}")

logger.info("Bayesian inference complete", num_samples=2000, num_chains=4)

## ArviZ Diagnostics

Assess MCMC convergence and posterior quality.

In [ ]:
import arviz as az

# Convert to InferenceData
idata = az.from_numpyro(result_bayes.mcmc)

# Summary statistics
summary = az.summary(idata, hdi_prob=0.95)
print("\nArviZ Summary:")
print(summary)

# Check convergence
r_hat_max = summary['r_hat'].max()
ess_bulk_min = summary['ess_bulk'].min()
ess_tail_min = summary['ess_tail'].min()

print(f"\nConvergence Diagnostics:")
print(f"  Max R-hat: {r_hat_max:.4f} (target: < 1.01)")
print(f"  Min ESS bulk: {ess_bulk_min:.0f} (target: > 400)")
print(f"  Min ESS tail: {ess_tail_min:.0f} (target: > 400)")

if r_hat_max > 1.01:
    logger.warning("Poor convergence detected", r_hat_max=r_hat_max)
else:
    logger.info("MCMC converged", r_hat_max=r_hat_max, ess_bulk_min=ess_bulk_min)

In [ ]:
# Trace plots (check mixing)
az.plot_trace(idata, compact=True, figsize=(12, 10))
plt.tight_layout()
plt.show()

In [ ]:
# Pair plot (correlations)
# Use actual parameter names from FluidityNonlocal model
param_names = list(model_nonlocal.parameters.keys())[:5]  # First 5 parameters

az.plot_pair(
    idata,
    var_names=param_names,
    kind='hexbin',
    divergences=True,
    figsize=(12, 12)
)
plt.tight_layout()
plt.show()
plt.close('all')

In [ ]:
# Forest plot (credible intervals)
az.plot_forest(idata, hdi_prob=0.95, figsize=(10, 8))
plt.tight_layout()
plt.show()

## Comparison: Nonlocal vs Local Model

Fit **FluidityLocal** to assess whether non-local effects are necessary.

In [ ]:
# Initialize local model (no spatial diffusion)
model_local = FluidityLocal()

# NLSQ fit
model_local.fit(gamma_dot, sigma, test_mode='flow_curve', method='scipy')

# Predictions
sigma_pred_local = model_local.predict(gamma_dot, test_mode='flow_curve')

# Compute fit quality
metrics_local = compute_fit_quality(sigma, np.array(sigma_pred_local).flatten())
r_squared_local = metrics_local['R2']

print(f"\nLocal Model R² = {r_squared_local:.6f}")
print(f"Nonlocal Model R² = {r_squared:.6f}")
print(f"Improvement: ΔR² = {r_squared - r_squared_local:.6f}")

# Overlay comparison
fig, ax = plt.subplots(figsize=(10, 6))
ax.loglog(gamma_dot, sigma, 'o', label='Data', alpha=0.7, markersize=6)
ax.loglog(gamma_dot, sigma_pred_nlsq, '-', label='Nonlocal (NLSQ)', linewidth=2.5)
ax.loglog(gamma_dot, sigma_pred_local, '--', label='Local (NLSQ)', linewidth=2)
ax.set_xlabel(r'Shear Rate $\dot{\gamma}$ (s$^{-1}$)')
ax.set_ylabel(r'Stress $\sigma$ (Pa)')
ax.set_title('Nonlocal vs Local Model Predictions')
ax.legend()
ax.grid(True, which='both', alpha=0.3)
plt.tight_layout()
plt.show()
plt.close('all')

logger.info(
    "Model comparison complete",
    local_R2=r_squared_local,
    nonlocal_R2=r_squared
)

## Shear Banding Analysis

Compute fluidity profile statistics to detect shear banding.

In [ ]:
# Extract cooperativity length from posterior
xi_samples = posterior['xi']
xi_median = float(jnp.median(xi_samples))
xi_hdi = intervals['xi']

print(f"\nCooperativity Length ξ:")
print(f"  Median: {xi_median*1e6:.2f} μm")
print(f"  95% HDI: [{xi_hdi[0]*1e6:.2f}, {xi_hdi[1]*1e6:.2f}] μm")
print(f"  Gap width: {model_nonlocal.gap_width*1e3:.2f} mm")
print(f"  Ratio ξ/h: {xi_median/model_nonlocal.gap_width:.4f}")

# Shear banding criterion
if xi_median / model_nonlocal.gap_width > 0.01:
    print("\n⚠ Non-local effects significant (ξ/h > 0.01) → potential shear banding")
else:
    print("\n✓ Local approximation valid (ξ/h < 0.01) → minimal shear banding")

# Note: Full fluidity profile f(y) would require spatial simulation
# For flow curve fitting, we use spatially-averaged response
# True banding detection requires startup/creep protocols with spatial resolution

## Save Results

Export NLSQ parameters, Bayesian posteriors, and diagnostics.

In [ ]:
# Create output directory
output_dir = Path('../outputs/fluidity/nonlocal/flow_curve')
output_dir.mkdir(parents=True, exist_ok=True)

# Save NLSQ parameters
params_file = output_dir / 'nlsq_parameters.txt'
with open(params_file, 'w') as f:
    f.write("NLSQ Fitted Parameters\n")
    f.write("=" * 40 + "\n\n")
    for name, value in params_nlsq.items():
        f.write(f"{name:12s}: {value:12.6e}\n")
    f.write(f"\nR² = {r_squared:.8f}\n")
    f.write(f"RMSE = {rmse:.6f} Pa\n")

# Save Bayesian summary
summary_file = output_dir / 'bayesian_summary.txt'
with open(summary_file, 'w') as f:
    f.write(summary.to_string())

# Save posterior samples (NetCDF format)
posterior_file = output_dir / 'posterior.nc'
idata.to_netcdf(posterior_file)

# Save comparison plot
fig, ax = plt.subplots(figsize=(10, 6))
ax.loglog(gamma_dot, sigma, 'o', label='Data', alpha=0.7, markersize=6)
ax.loglog(gamma_dot, sigma_pred_nlsq, '-', label='Nonlocal', linewidth=2.5)
ax.loglog(gamma_dot, sigma_pred_local, '--', label='Local', linewidth=2)
ax.set_xlabel(r'Shear Rate $\dot{\gamma}$ (s$^{-1}$)')
ax.set_ylabel(r'Stress $\sigma$ (Pa)')
ax.set_title('FluidityNonlocal vs FluidityLocal')
ax.legend()
ax.grid(True, which='both', alpha=0.3)
plt.tight_layout()
plt.savefig(output_dir / 'model_comparison.png', dpi=300, bbox_inches='tight')
plt.show()
plt.close('all')

print(f"\nResults saved to: {output_dir.absolute()}")
logger.info("Results saved", output_dir=str(output_dir.absolute()))

## Key Takeaways

### When Non-Local Effects Matter

1. **Cooperativity length ξ:**
   - $\xi/h < 0.01$: Local model sufficient (uniform fluidity)
   - $0.01 < \xi/h < 0.1$: Moderate non-local effects (weak banding)
   - $\xi/h > 0.1$: Strong non-local effects (pronounced shear banding)

2. **Protocol dependence:**
   - **Flow curves**: Steady-state averages $\implies$ local model often adequate
   - **Startup**: Transient banding $\implies$ nonlocal model critical
   - **Creep**: Viscosity bifurcation $\implies$ nonlocal reveals delayed yielding

3. **Material signatures:**
   - **Emulsions**: Moderate ξ (10-50 μm), weak-to-moderate banding
   - **Microgels**: Large ξ (50-200 μm), strong banding
   - **Colloidal glasses**: Small ξ (1-10 μm), nearly local behavior

4. **Computational cost:**
   - Local model: $O(N_{\dot{\gamma}})$ algebraic solve
   - Nonlocal model: $O(N_{\dot{\gamma}} \times N_y \times N_t)$ PDE solve
   - Use local model first; upgrade to nonlocal if $R^2$ improvement $> 0.05$

### Model Selection Workflow

```
1. Fit FluidityLocal → get R²_local
2. Fit FluidityNonlocal → get R²_nonlocal, ξ
3. If ΔR² > 0.05 AND ξ/h > 0.01:
     → Nonlocal effects significant, use FluidityNonlocal
   Else:
     → Local approximation valid, use FluidityLocal
```

### Next Steps

- **Startup simulations**: Reveal stress overshoot and band formation dynamics
- **Creep protocols**: Detect viscosity bifurcation (slow vs fast creep)
- **LAOS analysis**: Nonlinear stress-strain loops with spatial heterogeneity
- **Parameter identifiability**: Joint analysis of multiple protocols to constrain ξ